In [ ]:
import torch
import numpy as np
from model.snn import SNN
from sklearn.model_selection import StratifiedKFold
import pandas as pd

In [ ]:
cv_list = [1, 2, 3, 4, 5]

In [ ]:
for cv in cv_list:
    backbone_list = ['transformer', 'resnet', 'lstm']
    backbone = backbone_list[1]
    train_index = '100_cv{}'.format(cv)
    # 加载模型
    device = torch.device('cpu')
    model = SNN(701, 701, input_dim=512, feature_dim=256, backbone=backbone, r_layer_num=1, t_num_layers=1,
                l_num_layers=1)
    model = model.to(device)
    model_path = 'fsl_{}_{}_1024.pth'.format(backbone, train_index)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    support_df = pd.read_csv('few_P300_cv{}_100.csv'.format(cv))
    support_file_list = support_df['data_file'].to_numpy()
    support_label_list = support_df['label'].to_numpy()
    # 数据拆分器
    support_data_df = None
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(support_file_list, support_label_list):
        support_data_df = support_df.iloc[test_index]
        break

    query_data_df = pd.read_csv('test_R300_cv{}_3900.csv'.format(cv))

    correct_count = 0
    for q_i, query_data in query_data_df.iterrows():
        # start_time = time.time()

        q_data = np.loadtxt(query_data['data_file'], delimiter='\t', dtype=float).T
        q_data_y = q_data[1]
        q_data_y = np.expand_dims(q_data_y, axis=0)
        q_data_y = np.expand_dims(q_data_y, axis=0)
        q_data_y = torch.from_numpy(q_data_y).float()

        true_label = int(query_data['label'])

        sim_list, support_label_list = [], []
        for s_i, support_data in support_data_df.iterrows():
            s_data = np.loadtxt(support_data['data_file'], delimiter='\t', dtype=float).T
            s_data_y = s_data[1]
            s_data_y = np.expand_dims(s_data_y, axis=0)
            s_data_y = np.expand_dims(s_data_y, axis=0)
            s_data_y = torch.from_numpy(s_data_y).float()

            support_label = int(support_data['label'])
            support_label_list.append(support_label)

            output = model(s_data_y.to(device), q_data_y.to(device))
            sim_list.append(output.item())

        pred_label_index = sim_list.index(max(sim_list))
        # print(sim_list)

        if true_label == support_label_list[pred_label_index]:
            correct_count += 1
        print(true_label, support_label_list[pred_label_index])
        # end_time = time.time()
        # print("耗时: {:.6f}秒".format(end_time - start_time))

    print('cv:{}, acc:{:.8f}'.format(cv, correct_count / len(query_data_df)))
        